# Cancer classifier using a convolutional neuronal network

In [20]:
#import libraries
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Input
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD

### Define parameters

In [21]:
#The given pictures are of 150 px * 150px | 8 classes are given
img_size = (150, 150)
batch_size = 64
image_path = "Kather_texture_2016_image_tiles_5000"

### Generate images rescaled

In [22]:
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

train_gen = datagen.flow_from_directory(
    image_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical",
    subset="training"
)

val_gen = datagen.flow_from_directory(
    image_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical",
    subset="validation"
)

Found 4000 images belonging to 8 classes.
Found 1000 images belonging to 8 classes.


### Define the CNN model


In [23]:
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation="relu", input_shape=(150,150,3),padding="same"),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(64, (3,3), activation="relu", padding="same"),
    layers.BatchNormalization(),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(128, (3,3), activation="relu", padding="same"),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2,2)),

    layers.Flatten(),
    layers.Dropout(0.5),
    layers.Dense(128, activation="relu"),
    layers.Dense(train_gen.num_classes, activation="softmax")
])

opt = SGD(learning_rate=0.001, momentum=0.9)

model.compile(optimizer=opt,
              loss="categorical_crossentropy",
              metrics=["accuracy"])

### Train the model

In [ ]:
history = model.fit(
    train_gen,
    steps_per_epoch=len(train_gen),
    validation_data=val_gen,
    validation_steps=len(val_gen),
    epochs=15
)

Epoch 1/15


### Evaluation

In [ ]:
test_loss, test_acc = model.evaluate(val_gen)
print(f"Validation Accuracy: {test_acc:.2f}")

In [ ]:
pyplot.figure(figsize=(12, 4))

# plot loss
pyplot.subplot(2, 1, 1)
pyplot.title('Cross Entropy Loss')
pyplot.plot(history.history['loss'], color='blue', label='train')
pyplot.plot(history.history['val_loss'], color='orange', label='validation')
pyplot.ylim(0.0,1.0)
pyplot.legend()

# plot accuracy
pyplot.subplot(2,1,2)
pyplot.title('Classification Accuracy')
pyplot.plot(history.history['accuracy'], color='blue', label='train')
pyplot.plot(history.history['val_accuracy'], color='orange', label='validation')
pyplot.legend()
pyplot.ylim(0.0,1.0)

pyplot.tight_layout()
pyplot.show()